## 메뉴 전처리

In [7]:
pip install sentencepiece

Note: you may need to restart the kernel to use updated packages.


In [9]:
import numpy as np
import pandas as pd
import tensorflow as tf
import sentencepiece as spm
import csv
from tensorflow.keras.preprocessing.text import Tokenizer

In [12]:
train_df = pd.read_csv('C:\\Users\\lg\\Desktop\\DACON\\data\\train.csv', encoding='utf-8')
test_df = pd.read_csv('C:\\Users\\lg\\Desktop\\DACON\\data\\test.csv', encoding='utf-8')

df = pd.concat([train_df, test_df], axis=0).reset_index(drop=True)

columns = ['조식메뉴', '중식메뉴', '석식메뉴']
for col in columns :
    df[col] = df[col].str.replace('/', ' ')
    df[col] = df[col].str.replace(r'([<]{1}[ㄱ-힣\:\,\.\/\-A-Za-z 0-9]*[>]{1})', '')
    df[col] = df[col].str.replace(r'([＜]{1}[ㄱ-힣\:\,\.\/\-A-Za-z 0-9]*[＞]{1})', '')
    df[col] = df[col].str.replace(r'([(]{1}[ㄱ-힣\:\,\.\/\-A-Za-z 0-9]*[)]{1})', '')
    df[col] = df[col].str.replace(r'[ ]{2, }', ' ')
    df[col] = df[col].str.replace('\(New\)', '')
    df[col] = df[col].str.replace('\(NeW\)', '')
    df[col] = df[col].str.replace(r'[D]{1}', '')
    df[col] = df[col].str.replace(r'[S]{1}', '')
    df[col] = df[col].str.replace('\(쌀:국내산,돈육:국내', '')
    df[col] = df[col].str.replace('고추가루:중국산\)', '')
    df[col] = df[col].str.replace('*', ' ')
    df[col] = df[col].str.replace('[(]만두 고추 통계란[)]', '')
    df[col] = df[col].str.replace('[(]모둠튀김 양념장[)]', '')
    df[col] = df[col].apply(lambda x : x.strip())

corpus = pd.concat([df.조식메뉴, df.중식메뉴, df.석식메뉴], axis=0).reset_index(drop=True)
corpus

0       모닝롤 찐빵  우유 두유 주스 계란후라이  호두죽 쌀밥  된장찌개  쥐어채무침  포기김치
1       모닝롤 단호박샌드  우유 두유 주스 계란후라이  팥죽 쌀밥  호박젓국찌개  시래기조...
2       모닝롤 베이글  우유 두유 주스 계란후라이  표고버섯죽 쌀밥  콩나물국  느타리호박...
3       모닝롤 토마토샌드  우유 두유 주스 계란후라이  닭죽 쌀밥  근대국  멸치볶음  포기김치
4        모닝롤 와플  우유 두유 주스 계란후라이  쇠고기죽 쌀밥  재첩국  방풍나물  포기김치
                              ...                        
3760                       흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기
3761              추가밥 짬뽕 생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치
3762              단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치
3763                   흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음
3764                 흑미밥 맑은버섯국 매운사태조림 춘권 타르타르 열무나물무침 포기김치
Length: 3765, dtype: object

In [13]:
print(corpus.isnull().values.any()) # Null 값 존재하는지 확인
print(corpus)

False
0       모닝롤 찐빵  우유 두유 주스 계란후라이  호두죽 쌀밥  된장찌개  쥐어채무침  포기김치
1       모닝롤 단호박샌드  우유 두유 주스 계란후라이  팥죽 쌀밥  호박젓국찌개  시래기조...
2       모닝롤 베이글  우유 두유 주스 계란후라이  표고버섯죽 쌀밥  콩나물국  느타리호박...
3       모닝롤 토마토샌드  우유 두유 주스 계란후라이  닭죽 쌀밥  근대국  멸치볶음  포기김치
4        모닝롤 와플  우유 두유 주스 계란후라이  쇠고기죽 쌀밥  재첩국  방풍나물  포기김치
                              ...                        
3760                       흑미밥 돈육고추장찌개 갈치구이 김치전 취나물무침 깍두기
3761              추가밥 짬뽕 생면 수제찹쌀꿔바로우 메추리알곤약장조림 단무지무침 포기김치
3762              단호박카레라이스 시금치된장국 소떡소떡 파프리카해초무침 감귤쥬스 포기김치
3763                   흑미밥 어묵매운탕 쇠고기숙주볶음 채소계란찜 쑥갓생무침 김치볶음
3764                 흑미밥 맑은버섯국 매운사태조림 춘권 타르타르 열무나물무침 포기김치
Length: 3765, dtype: object


In [14]:
with open('Menu.txt', 'w', encoding='utf8') as f:
    f.write('\n'.join(corpus))

In [26]:
spm.SentencePieceTrainer.Train('--input=Menu.txt --model_prefix=Menu --vocab_size=500 --model_type=bpe --max_sentence_length=9999')
vocab_list_500 = pd.read_csv('Menu.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list_500.to_csv('vocab_list_500.csv', index=True, encoding='utf-8-sig')
vocab_list_500.sample(10)

,0,1
277,회,-274
152,꽃,-149
279,발,-276
443,맥,-440
201,등,-198
44,주,-41
189,커,-186
274,산,-271
206,홍,-203
245,루,-242


In [27]:
spm.SentencePieceTrainer.Train('--input=Menu.txt --model_prefix=Menu --vocab_size=1000 --model_type=bpe --max_sentence_length=9999')
vocab_list_1000 = pd.read_csv('Menu.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list_1000.to_csv('vocab_list_1000.csv', index=True, encoding='utf-8-sig')
vocab_list_1000.sample(10)

,0,1
193,▁참나물,-190
687,까,-684
117,▁청,-114
492,▁쇠고기무국,-489
158,미역,-155
966,섬,-963
168,커리,-165
290,▁아삭,-287
95,새우,-92
949,T,-946


In [28]:
spm.SentencePieceTrainer.Train('--input=Menu.txt --model_prefix=Menu --vocab_size=2000 --model_type=bpe --max_sentence_length=9999')
vocab_list_2000 = pd.read_csv('Menu.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list_2000.to_csv('vocab_list_2000.csv', index=True, encoding='utf-8-sig')
vocab_list_2000.sample(10)

,0,1
1168,감자채볶음,-1165
1549,채,-1546
697,▁표고,-694
717,▁햄치즈샌드,-714
488,▁무피클,-485
904,▁참나물땅콩,-901
1092,▁연근땅콩조림,-1089
972,▁유부주머니,-969
1449,▁로제파스타,-1446
1706,홍,-1703


In [32]:
spm.SentencePieceTrainer.Train('--input=Menu.txt --model_prefix=Menu --vocab_size=3000 --model_type=bpe --max_sentence_length=9999')
vocab_list_3000 = pd.read_csv('Menu.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list_3000.to_csv('vocab_list_3000.csv', index=True, encoding='utf-8-sig')
vocab_list_3000.sample(10)

,0,1
1579,▁제육,-1576
916,크림,-913
542,▁어묵볶음,-539
2381,고추조림,-2378
2248,골식,-2245
1573,▁녹차,-1570
12,무침,-9
2556,고,-2553
2357,▁수제오,-2354
233,▁황태,-230


In [31]:
spm.SentencePieceTrainer.Train('--input=Menu.txt --model_prefix=Menu --vocab_size=1500 --model_type=bpe --max_sentence_length=9999')
vocab_list_1500 = pd.read_csv('Menu.vocab', sep='\t', header=None, quoting=csv.QUOTE_NONE)
vocab_list_1500.to_csv('vocab_list_1500.csv', index=True, encoding='utf-8-sig')
vocab_list_1500.sample(10)


,0,1
747,▁김치전,-744
440,라이스,-437
1034,김,-1031
146,장조림,-143
209,▁모듬,-206
101,▁김,-98
401,위치,-398
1380,색,-1377
1477,향,-1474
345,▁모둠,-342


In [20]:
sp = spm.SentencePieceProcessor()
vocab_file = "Menu.model"
sp.load(vocab_file)

True